# Importar librerias

## Instalacion librerias

In [ ]:
!pip install gdown

In [ ]:
import gdown
import pandas as pd

# Importación dataset

## Instalacion de gdown


In [ ]:
url = 'https://drive.google.com/uc?id=1UMmO08jDRoOPyOffXdqWRy5TIw8GjtjU'

output = 'Crime_Data_from_2020_to_Present.csv'


gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1UMmO08jDRoOPyOffXdqWRy5TIw8GjtjU
From (redirected): https://drive.google.com/uc?id=1UMmO08jDRoOPyOffXdqWRy5TIw8GjtjU&confirm=t&uuid=1cda8f61-b8ed-4cf7-a37c-2f5df9983f49
To: /content/Crime_Data_from_2020_to_Present.csv
100%|██████████| 251M/251M [00:01<00:00, 130MB/s]


'Crime_Data_from_2020_to_Present.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(output)
df.head(10)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,18,Southeast,1826,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,9900 COMPTON AV,NaN,33.9467,-118.2463
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1,Central,182,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 S GRAND AV,NaN,34.0415,-118.2620
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3,Southwest,303,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,2500 S SYCAMORE AV,NaN,34.0335,-118.3537
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,13,Newton,1375,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1300 E 57TH ST,NaN,33.9911,-118.2521
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19,Mission,1974,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,9000 CEDROS AV,NaN,34.2336,-118.4535


In [ ]:
print(df.head(10))

       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0  190326475  03/01/2020 12:00:00 AM  03/01/2020 12:00:00 AM      2130     7   
1  200106753  02/09/2020 12:00:00 AM  02/08/2020 12:00:00 AM      1800     1   
2  200320258  11/11/2020 12:00:00 AM  11/04/2020 12:00:00 AM      1700     3   
3  200907217  05/10/2023 12:00:00 AM  03/10/2020 12:00:00 AM      2037     9   
4  220614831  08/18/2022 12:00:00 AM  08/17/2020 12:00:00 AM      1200     6   
5  231808869  04/04/2023 12:00:00 AM  12/01/2020 12:00:00 AM      2300    18   
6  230110144  04/04/2023 12:00:00 AM  07/03/2020 12:00:00 AM       900     1   
7  220314085  07/22/2022 12:00:00 AM  05/12/2020 12:00:00 AM      1110     3   
8  231309864  04/28/2023 12:00:00 AM  12/09/2020 12:00:00 AM      1400    13   
9  211904005  12/31/2020 12:00:00 AM  12/31/2020 12:00:00 AM      1220    19   

   AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0   Wilshire          784         1     510   
1    Central          182

In [ ]:
df.iloc[:,:14].head()


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,M,O
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,X,X
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H


In [ ]:
df.iloc[:,14:].head()

,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,101.0,STREET,NaN,NaN,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,128.0,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,405.0,CLOTHING STORE,NaN,NaN,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,102.0,SIDEWALK,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [ ]:
df.iloc[:,14:].tail()

,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
986495,207.0,BAR/COCKTAIL/NIGHTCLUB,NaN,NaN,IC,Invest Cont,341.0,NaN,NaN,NaN,3700 WILSHIRE BL,NaN,34.0617,-118.3066
986496,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,4000 W 23RD ST,NaN,34.0362,-118.3284
986497,503.0,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,745.0,NaN,NaN,NaN,1300 W SUNSET BL,NaN,34.0685,-118.2460
986498,102.0,SIDEWALK,308.0,STICK,IC,Invest Cont,230.0,NaN,NaN,NaN,FLOWER ST,JEFFERSON BL,34.0215,-118.2868
986499,108.0,PARKING LOT,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,6900 VESPER AV,NaN,34.1961,-118.4510


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           986500 non-null  int64  
 1   Date Rptd       986500 non-null  object 
 2   DATE OCC        986500 non-null  object 
 3   TIME OCC        986500 non-null  int64  
 4   AREA            986500 non-null  int64  
 5   AREA NAME       986500 non-null  object 
 6   Rpt Dist No     986500 non-null  int64  
 7   Part 1-2        986500 non-null  int64  
 8   Crm Cd          986500 non-null  int64  
 9   Crm Cd Desc     986500 non-null  object 
 10  Mocodes         840065 non-null  object 
 11  Vict Age        986500 non-null  int64  
 12  Vict Sex        846925 non-null  object 
 13  Vict Descent    846914 non-null  object 
 14  Premis Cd       986486 non-null  float64
 15  Premis Desc     985915 non-null  object 
 16  Weapon Used Cd  326368 non-null  float64
 17  Weapon Des

In [ ]:
df.describe()

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
count,9.865000e+05,986500.000000,986500.000000,986500.000000,986500.000000,986500.000000,986500.000000,986486.000000,326368.000000,986489.000000,68912.000000,2310.000000,64.00000,986500.000000,986500.000000
mean,2.198270e+08,1339.086392,10.698086,1116.245129,1.403666,500.782398,29.045177,306.054803,363.846008,500.538333,958.162091,984.198701,991.21875,33.996033,-118.083281
std,1.299240e+07,651.457178,6.108873,611.002343,0.490632,206.092913,21.976666,219.156166,123.699000,205.891829,110.250287,51.495991,27.06985,1.633543,5.661853
min,8.170000e+02,1.000000,1.000000,101.000000,1.000000,110.000000,-4.000000,101.000000,101.000000,110.000000,210.000000,310.000000,821.00000,0.000000,-118.667600
25%,2.106103e+08,900.000000,5.000000,587.000000,1.000000,331.000000,0.000000,101.000000,311.000000,331.000000,998.000000,998.000000,998.00000,34.014600,-118.430500
50%,2.208174e+08,1420.000000,11.000000,1141.000000,1.000000,442.000000,30.000000,203.000000,400.000000,442.000000,998.000000,998.000000,998.00000,34.058900,-118.322500
75%,2.310049e+08,1900.000000,16.000000,1617.000000,2.000000,626.000000,44.000000,501.000000,400.000000,626.000000,998.000000,998.000000,998.00000,34.164900,-118.273900
max,2.499289e+08,2359.000000,21.000000,2199.000000,2.000000,956.000000,120.000000,976.000000,516.000000,956.000000,999.000000,999.000000,999.00000,34.334300,0.000000


In [ ]:
# Exploración de primeros 50 valores únicos solo para variables categóricas
for col in df.select_dtypes(include='object').columns:
    print(f"--- {col} ---")
    print(df[col].dropna().unique()[:50])  # Primeros 50 valores únicos
    print("\n")



--- Date Rptd ---
['03/01/2020 12:00:00 AM' '02/09/2020 12:00:00 AM'
 '11/11/2020 12:00:00 AM' '05/10/2023 12:00:00 AM'
 '08/18/2022 12:00:00 AM' '04/04/2023 12:00:00 AM'
 '07/22/2022 12:00:00 AM' '04/28/2023 12:00:00 AM'
 '12/31/2020 12:00:00 AM' '01/21/2022 12:00:00 AM'
 '04/12/2022 12:00:00 AM' '01/05/2023 12:00:00 AM'
 '05/06/2022 12:00:00 AM' '06/01/2023 12:00:00 AM'
 '02/03/2023 12:00:00 AM' '12/24/2023 12:00:00 AM'
 '11/27/2020 12:00:00 AM' '09/20/2022 12:00:00 AM'
 '02/03/2022 12:00:00 AM' '04/25/2023 12:00:00 AM'
 '03/21/2023 12:00:00 AM' '06/04/2023 12:00:00 AM'
 '09/09/2020 12:00:00 AM' '08/03/2023 12:00:00 AM'
 '02/02/2022 12:00:00 AM' '02/23/2022 12:00:00 AM'
 '05/03/2020 12:00:00 AM' '03/20/2021 12:00:00 AM'
 '04/28/2022 12:00:00 AM' '02/27/2023 12:00:00 AM'
 '01/17/2023 12:00:00 AM' '05/09/2023 12:00:00 AM'
 '01/19/2022 12:00:00 AM' '09/06/2022 12:00:00 AM'
 '09/27/2021 12:00:00 AM' '10/22/2021 12:00:00 AM'
 '10/10/2022 12:00:00 AM' '03/28/2023 12:00:00 AM'
 '10/25/2023 

In [ ]:
# Vamos a renombrar las columnas para hacerlas más comprensibles, se trata de un cambio en el Diccionario
rename_columns = {
    'DR_NO': 'id_report',
    'Date Rptd': 'date_report',
    'DATE OCC': 'date_ocurrance',
    'TIME OCC': 'time_ocurrance',
    'AREA': 'area_code',
    'AREA NAME': 'area_name',
    'Rpt Dist No': 'district',
    'Part 1-2': 'crime_severity',
    'Crm Cd': 'crime_code',
    'Crm Cd Desc': 'crime_desc',
    'Mocodes': 'crime_mo',
    'Vict Age': 'age_victim',
    'Vict Sex': 'sex_victim',
    'Vict Descent': 'descent_victim',
    'Premis Cd': 'loc_code',
    'Premis Desc': 'loc_desc',
    'Weapon Used Cd': 'weapon_code',
    'Weapon Desc': 'weapon_desc',
    'Status': 'case_status_code',
    'Status Desc': 'case_status_desc',
    'Crm Cd 1': 'crime_code_1',
    'Crm Cd 2': 'crime_code_2',
    'Crm Cd 3': 'crime_code_3',
    'Crm Cd 4': 'crime_code_4',
    'LOCATION': 'location',
    'Cross Street': 'cross_street',
    'LAT': 'latitude',
    'LON': 'longitude'
}

# Renombrar las columnas
df.rename(columns=rename_columns, inplace=True)

# Confirmar el cambio
df.head()


,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [ ]:
# Variables a explorar
columns_to_explore = [
    'area_code',
    'area_name',
    'district',
    'crime_severity',
    'crime_code',
    'crime_desc',
    'sex_victim',
    'descent_victim',
    'loc_code',
    'loc_desc',
    'weapon_code',
    'weapon_desc',
    'case_status_code',
    'case_status_desc'
]

# Mostrar TODOS los valores únicos para esas columnas
for col in columns_to_explore:
    if col in df.columns:
        print(f"--- {col} ---")
        print(sorted(df[col].dropna().unique()))
        print("\n")
    else:
        print(f"Column {col} not found in the dataframe.\n")


--- area_code ---
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21)]


--- area_name ---
['77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire']


--- district ---
[np.int64(101), np.int64(105), np.int64(109), np.int64(111), np.int64(112), np.int64(118), np.int64(119), np.int64(121), np.int64(122), np.int64(123), np.int64(124), np.int64(127), np.int64(128), np.int64(129), np.int64(131), np.int64(132), np.int64(133), np.int64(134), np.int64(135), np.int64(138), np.int64(139), np.int64(141), np.int64(142), np.int64(143), np.int64(144), np.int64(145), np.int6

## **Vamos a trabajar en investigar la relación de códigos varios en el dataset. **

# 📄 Unique Values Summary

## area_code
`1, 2, 3, ..., 21`

## area_name
`'77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire'`

## district
`101, 105, 109, 111, ..., 2199`  
(Muchos valores únicos de distrito, códigos de zonas específicas)

## crime_severity
`1, 2`

## crime_code
`110, 113, 121, 122, 210, 220, 230, 231, ..., 956`

## crime_desc
`'ARSON', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER', 'BATTERY - SIMPLE ASSAULT', 'BIKE - STOLEN', 'BURGLARY', 'BURGLARY FROM VEHICLE', ..., 'WEAPONS POSSESSION/BOMBING'`
(Más de 100 tipos de crímenes diferentes)

## crime_mo
`Códigos tipo '0344 1822 1402', '0930', '0913 2024' etc.`  
(Modus Operandi combinados por código)

## sex_victim
`'-', 'F', 'H', 'M', 'X'`
(Indica sexo: Femenino, Masculino, Indeterminado, No reportado)

## descent_victim
`'-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z'`
(Origen étnico de la víctima)

## loc_code
`101.0, 102.0, 103.0, ..., 976.0`
(Muchos códigos de localización)

## loc_desc
`'STREET', 'BUS STOP', 'PARKING LOT', 'SINGLE FAMILY DWELLING', 'SCHOOL', 'HOSPITAL', 'HOTEL', 'MARKET', ..., 'YARD (RESIDENTIAL/BUSINESS)'`
(Lugares donde ocurrieron los crímenes)

## weapon_code
`101.0, 102.0, ..., 516.0`

## weapon_desc
`'HAND GUN', 'KNIFE WITH BLADE 6INCHES OR LESS', 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)', 'UNKNOWN FIREARM', 'VERBAL THREAT', ...`
(Descripción de armas usadas)

## case_status_code
`'AA', 'AO', 'CC', 'IC', 'JA', 'JO'`
(Códigos de estatus del caso)

## case_status_desc
`'Adult Arrest', 'Adult Other', 'Invest Cont', 'Juv Arrest', 'Juv Other', 'UNK'`
(Descripción del estatus del caso)


##1. Investigación de las áreas.

In [ ]:
# Primero aseguramos que district sea string para extraer area_code
df['district'] = df['district'].astype(str)

# Sacamos el primer dígito como área (puedes usar 1 o 2 dígitos según precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Mapeo de area_code a area_name
area_mapping = {
    1: '77th Street',
    2: 'Central',
    3: 'Devonshire',
    4: 'Foothill',
    5: 'Harbor',
    6: 'Hollenbeck',
    7: 'Hollywood',
    8: 'Mission',
    9: 'N Hollywood',
    10: 'Newton',
    11: 'Northeast',
    12: 'Olympic',
    13: 'Pacific',
    14: 'Rampart',
    15: 'Southeast',
    16: 'Southwest',
    17: 'Topanga',
    18: 'Van Nuys',
    19: 'West LA',
    20: 'West Valley',
    21: 'Wilshire'
}

# Asignamos nombre al área
df['area_name'] = df['area_code'].map(area_mapping)

# Verificamos si el mapeo está correcto
print(df[['district', 'area_code', 'area_name']].head())

# Contar registros por area_name
print(df['area_name'].value_counts())


  district  area_code    area_name
0      784          7    Hollywood
1      182          1  77th Street
2      356          3   Devonshire
3      964          9  N Hollywood
4      666          6   Hollenbeck
area_name
77th Street    531859
Central        135803
Devonshire      56259
Hollenbeck      51510
Hollywood       47272
Mission         44848
N Hollywood     42059
Harbor          40603
Foothill        36287
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Definimos el listado completo de áreas (en el orden correcto)
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# Hacemos el conteo de registros por area_name
area_counts = df['area_name'].value_counts()

# Aseguramos que todos los nombres estén incluidos (incluso los que faltan)
area_counts = area_counts.reindex(area_names, fill_value=0)

# Ahora, buscamos las áreas con 0 crímenes
areas_sin_crimen = area_counts[area_counts == 0]

# Mostrar áreas sin crímenes
print("Áreas donde no hubo crímenes registrados:")
print(areas_sin_crimen)

Áreas donde no hubo crímenes registrados:
area_name
Newton         0
Northeast      0
Olympic        0
Pacific        0
Rampart        0
Southeast      0
Southwest      0
Topanga        0
Van Nuys       0
West LA        0
West Valley    0
Wilshire       0
Name: count, dtype: int64


In [ ]:
# Definimos las 21 áreas oficiales
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# 1. Conteo de crímenes por área
area_counts = df['area_name'].value_counts()

# 2. Asegurarnos que todas las áreas estén (rellenar con 0 si no existen)
area_counts = area_counts.reindex(area_names, fill_value=0)

# 3. Detectar áreas sin crímenes
areas_sin_crimen = area_counts[area_counts == 0].index.tolist()

print("Áreas donde no hubo crímenes:")
print(areas_sin_crimen)

# 4. Filtrar el df para filas con esas áreas (debería estar vacío, o muy pequeño si algo raro pasó)
df_sin_crimenes = df[df['area_name'].isin(areas_sin_crimen)]

# 5. Mostrar las filas
print(df_sin_crimenes)


Áreas donde no hubo crímenes:
['Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire']
Empty DataFrame
Columns: [id_report, date_report, date_ocurrance, time_ocurrance, area_code, area_name, district, crime_severity, crime_code, crime_desc, crime_mo, age_victim, sex_victim, descent_victim, loc_code, loc_desc, weapon_code, weapon_desc, case_status_code, case_status_desc, crime_code_1, crime_code_2, crime_code_3, crime_code_4, location, cross_street, latitude, longitude, crime_ucr, area_valid]
Index: []

[0 rows x 30 columns]


Resumen:
9 áreas sí tienen crímenes (las que viste antes: 77th Street, Central, Devonshire, etc.).

12 áreas no tienen crímenes en tus datos actuales.

No hay errores de mapeo en district, area_code ni area_name.

## 2. Investigación Distritos.

In [ ]:
# Mostrar los valores únicos de la columna 'district'
if 'district' in df.columns:
    print("--- district ---")
    print(sorted(df['district'].dropna().unique()))
    print("\n")
else:
    print("Column 'district' not found in the dataframe.\n")

--- district ---
[np.int64(101), np.int64(105), np.int64(109), np.int64(111), np.int64(112), np.int64(118), np.int64(119), np.int64(121), np.int64(122), np.int64(123), np.int64(124), np.int64(127), np.int64(128), np.int64(129), np.int64(131), np.int64(132), np.int64(133), np.int64(134), np.int64(135), np.int64(138), np.int64(139), np.int64(141), np.int64(142), np.int64(143), np.int64(144), np.int64(145), np.int64(146), np.int64(147), np.int64(148), np.int64(151), np.int64(152), np.int64(153), np.int64(154), np.int64(155), np.int64(156), np.int64(157), np.int64(158), np.int64(159), np.int64(161), np.int64(162), np.int64(163), np.int64(164), np.int64(165), np.int64(166), np.int64(171), np.int64(174), np.int64(176), np.int64(181), np.int64(182), np.int64(185), np.int64(191), np.int64(192), np.int64(195), np.int64(201), np.int64(202), np.int64(203), np.int64(204), np.int64(205), np.int64(211), np.int64(212), np.int64(213), np.int64(215), np.int64(216), np.int64(217), np.int64(218), np.int6

In [ ]:
# Asegurar que 'district' es string para extraer prefijos fácilmente
df['district'] = df['district'].astype(str)

# Extraer primer dígito (o primeros dos dígitos si quieres más precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Opcionalmente, si quieres extraer dos primeros dígitos:
# df['area_code'] = df['district'].str[:2].astype(int)

# Definir los áreas válidas (1 al 21)
valid_areas = list(range(1, 22))

# Marcar si es un área válida
df['area_valid'] = df['area_code'].isin(valid_areas)

# Verificar resultados
print(df['area_valid'].value_counts())

# Ver ejemplos
print(df[['district', 'area_code', 'area_valid']].head())


area_valid
True    986500
Name: count, dtype: int64
  district  area_code  area_valid
0      784          7        True
1      182          1        True
2      356          3        True
3      964          9        True
4      666          6        True


In [ ]:
# Contar cuántos registros NO son válidos
invalid_count = (~df['area_valid']).sum()

print(f"Cantidad de registros con area_valid == False: {invalid_count}")


Cantidad de registros con area_valid == False: 0


##3. Relación entre crime_code y crime_desc

Los crime_code y crime_desc que tienes provienen directamente de los reportes oficiales del LAPD (Los Angeles Police Department).

Cada "crime_code" es un tipo de crimen específico, basado en su severidad y en las leyes de California.

No es una clasificación inventada, es estándar dentro del departamento, usado para categorizar las denuncias.

In [ ]:
# Comprobar si un mismo crime_code tiene varias descripciones
crime_mapping = df[['crime_code', 'crime_desc']].drop_duplicates()

# Verificar duplicados de crime_code
duplicates = crime_mapping.duplicated(subset=['crime_code'], keep=False)
crime_mapping[duplicates]

,crime_code,crime_desc


Si no salen filas, la relación es uno a uno ✅.

In [ ]:
# Crear tabla resumen de códigos de crimen y su descripción
crime_table = df[['crime_code', 'crime_desc']].drop_duplicates().sort_values('crime_code')

# Mostrar la tabla
crime_table.reset_index(drop=True, inplace=True)
crime_table


,crime_code,crime_desc
0,110,CRIMINAL HOMICIDE
1,113,"MANSLAUGHTER, NEGLIGENT"
2,121,"RAPE, FORCIBLE"
3,122,"RAPE, ATTEMPTED"
4,210,ROBBERY
...,...,...
135,949,ILLEGAL DUMPING
136,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $..."
137,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &..."
138,954,CONTRIBUTING


In [ ]:
crime_table.to_csv('crime_code_description.csv', index=False)

Vamos a generar una agrupación de todos estos delitos en base al standard UCR (Uniform Crime Reporting)

In [ ]:
import pandas as pd

# Suponemos que tienes un DataFrame llamado 'df' con las columnas 'crime_code' y 'crime_desc'

# Mapeo de crime_codes a categorías UCR
ucr_mapping = {
    # Part I - Violent Crimes
    110: 'HOMICIDE',
    113: 'MANSLAUGHTER',
    121: 'RAPE',
    122: 'ATTEMPTED RAPE',
    815: 'SEXUAL PENETRATION',
    820: 'ORAL COPULATION',
    821: 'SODOMY',
    210: 'ROBBERY',
    220: 'ROBBERY ATTEMPTED',
    230: 'AGGRAVATED ASSAULT',
    231: 'AGGRAVATED ASSAULT',
    235: 'AGGRAVATED ASSAULT',
    236: 'AGGRAVATED ASSAULT',
    250: 'AGGRAVATED ASSAULT',
    251: 'AGGRAVATED ASSAULT',
    761: 'AGGRAVATED ASSAULT',
    926: 'AGGRAVATED ASSAULT',
    435: 'SIMPLE ASSAULT',
    436: 'SIMPLE ASSAULT',
    437: 'SIMPLE ASSAULT',
    622: 'SIMPLE ASSAULT',
    623: 'SIMPLE ASSAULT',
    624: 'SIMPLE ASSAULT',
    625: 'SIMPLE ASSAULT',
    626: 'SIMPLE ASSAULT',
    627: 'SIMPLE ASSAULT',
    647: 'SIMPLE ASSAULT',
    763: 'SIMPLE ASSAULT',
    928: 'SIMPLE ASSAULT',
    930: 'SIMPLE ASSAULT',

    # Part I - Property Crimes
    310: 'BURGLARY',
    320: 'BURGLARY',
    510: 'MOTOR VEHICLE THEFT',
    520: 'MOTOR VEHICLE THEFT',
    433: 'MOTOR VEHICLE THEFT',
    330: 'THEFT FROM VEHICLE',
    331: 'THEFT FROM VEHICLE',
    410: 'THEFT FROM VEHICLE',
    420: 'THEFT FROM VEHICLE',
    421: 'THEFT FROM VEHICLE',
    350: 'PERSONAL THEFT',
    351: 'PERSONAL THEFT',
    352: 'PERSONAL THEFT',
    353: 'PERSONAL THEFT',
    450: 'PERSONAL THEFT',
    451: 'PERSONAL THEFT',
    452: 'PERSONAL THEFT',
    453: 'PERSONAL THEFT',
    341: 'OTHER THEFT',
    343: 'OTHER THEFT',
    345: 'OTHER THEFT',
    440: 'OTHER THEFT',
    441: 'OTHER THEFT',
    442: 'OTHER THEFT',
    443: 'OTHER THEFT',
    444: 'OTHER THEFT',
    445: 'OTHER THEFT',
    470: 'OTHER THEFT',
    471: 'OTHER THEFT',
    472: 'OTHER THEFT',
    473: 'OTHER THEFT',
    474: 'OTHER THEFT',
    475: 'OTHER THEFT',
    480: 'OTHER THEFT',
    485: 'OTHER THEFT',
    487: 'OTHER THEFT',
    491: 'OTHER THEFT',

    # Part II - Other Crimes
    500: 'DRUG OFFENSES',
    501: 'DRUG OFFENSES',
    502: 'DRUG OFFENSES',
    503: 'DRUG OFFENSES',
    504: 'DRUG OFFENSES',
    505: 'DRUG OFFENSES',
    506: 'DRUG OFFENSES',
    600: 'SIMPLE ASSAULT',
    601: 'SIMPLE ASSAULT',
    602: 'SIMPLE ASSAULT',
    603: 'SIMPLE ASSAULT',
    700: 'VANDALISM',
    701: 'VANDALISM',
    702: 'WEAPONS OFFENSE',
    703: 'WEAPONS OFFENSE',
    704: 'WEAPONS OFFENSE',
    705: 'WEAPONS OFFENSE',
    800: 'SEX OFFENSES (NON-RAPE)',
    801: 'SEX OFFENSES (NON-RAPE)',
    802: 'SEX OFFENSES (NON-RAPE)',
    803: 'SEX OFFENSES (NON-RAPE)',
    900: 'OTHER OFFENSES',
    901: 'OTHER OFFENSES',
    902: 'OTHER OFFENSES',
    903: 'OTHER OFFENSES',
    904: 'OTHER OFFENSES',
    905: 'OTHER OFFENSES',
    906: 'OTHER OFFENSES',
    907: 'OTHER OFFENSES'
}

# Crear nueva columna 'crime_ucr' basada en el mapping
df['crime_ucr'] = df['crime_code'].map(ucr_mapping).fillna('UNKNOWN')

# Si quieres ver los resultados
print(df[['crime_code', 'crime_desc', 'crime_ucr']].head())


   crime_code                                crime_desc            crime_ucr
0         510                          VEHICLE - STOLEN  MOTOR VEHICLE THEFT
1         330                     BURGLARY FROM VEHICLE   THEFT FROM VEHICLE
2         480                             BIKE - STOLEN          OTHER THEFT
3         343  SHOPLIFTING-GRAND THEFT ($950.01 & OVER)          OTHER THEFT
4         354                         THEFT OF IDENTITY              UNKNOWN


In [ ]:
df.columns

Index(['id_report', 'date_report', 'date_ocurrance', 'time_ocurrance',
       'area_code', 'area_name', 'district', 'crime_severity', 'crime_code',
       'crime_desc', 'crime_mo', 'age_victim', 'sex_victim', 'descent_victim',
       'loc_code', 'loc_desc', 'weapon_code', 'weapon_desc',
       'case_status_code', 'case_status_desc', 'crime_code_1', 'crime_code_2',
       'crime_code_3', 'crime_code_4', 'location', 'cross_street', 'latitude',
       'longitude', 'crime_ucr'],
      dtype='object')

## 4. Investigación de Sexo.

In [ ]:
# 1. Primero, corregimos H -> M
df['sex_victim'] = df['sex_victim'].replace('H', 'M')

# 2. Definimos la lista válida final
valores_validos_sexo = ['-', 'F', 'M', 'X']

# 3. Comprobamos de nuevo si queda todo limpio
print("Valores únicos en sex_victim después de la corrección:")
print(df['sex_victim'].unique())

# 4. Opcional: identificar si queda algo inválido
valores_invalidos = df[~df['sex_victim'].isin(valores_validos_sexo)]

print(f"\nCantidad de registros con sex_victim inválido después de corregir: {len(valores_invalidos)}")


Valores únicos en sex_victim después de la corrección:
['M' 'X' 'F' nan '-']

Cantidad de registros con sex_victim inválido después de corregir: 139575


In [30]:
# Reemplazar nulos por '-'
df['sex_victim'] = df['sex_victim'].fillna('-')

# Volver a verificar
print("Valores únicos en sex_victim después de rellenar NaN:")
print(df['sex_victim'].unique())

Valores únicos en sex_victim después de rellenar NaN:
['M' 'X' 'F' '-']


In [31]:
# 1. Conteo de sex_victim
conteo_sexo = df['sex_victim'].value_counts()

# 2. Porcentaje de sex_victim
porcentaje_sexo = df['sex_victim'].value_counts(normalize=True) * 100

# 3. Mostrar resultados
resultado = pd.DataFrame({
    'Cantidad': conteo_sexo,
    'Porcentaje (%)': porcentaje_sexo.round(2)
})

print(resultado)

            Cantidad  Porcentaje (%)
sex_victim                          
M             398060           40.35
F             353688           35.85
-             139576           14.15
X              95176            9.65


### id_report

In [ ]:
df['id_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: id_report
Non-Null Count   Dtype
--------------   -----
986500 non-null  int64
dtypes: int64(1)
memory usage: 7.5 MB


In [ ]:
df.iloc[:, :5].head(10)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,18
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,13
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19


In [ ]:
df.info()

### date_report

### date_ocurrance

### time_ocurrance

### area_code

### area_name

### age_victim

### location

### cross_street

### latitude

### longitude

### Todo

In [ ]:
features_explore_JA = ['id_report','date_report','date_ocurrance','time_ocurrance','area_code','area_name','age_victim','location','cross_street','latitude','longitude']

In [ ]:
df[features_explore_JA].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_report       986500 non-null  int64  
 1   date_report     986500 non-null  object 
 2   date_ocurrance  986500 non-null  object 
 3   time_ocurrance  986500 non-null  int64  
 4   area_code       986500 non-null  int64  
 5   area_name       986500 non-null  object 
 6   age_victim      986500 non-null  int64  
 7   location        986500 non-null  object 
 8   cross_street    152270 non-null  object 
 9   latitude        986500 non-null  float64
 10  longitude       986500 non-null  float64
dtypes: float64(2), int64(4), object(5)
memory usage: 82.8+ MB


In [ ]:
df[features_explore_JA].describe()

,id_report,time_ocurrance,area_code,age_victim,latitude,longitude
count,9.865000e+05,986500.000000,986500.000000,986500.000000,986500.000000,986500.000000
mean,2.198270e+08,1339.086392,10.698086,29.045177,33.996033,-118.083281
std,1.299240e+07,651.457178,6.108873,21.976666,1.633543,5.661853
min,8.170000e+02,1.000000,1.000000,-4.000000,0.000000,-118.667600
25%,2.106103e+08,900.000000,5.000000,0.000000,34.014600,-118.430500
50%,2.208174e+08,1420.000000,11.000000,30.000000,34.058900,-118.322500
75%,2.310049e+08,1900.000000,16.000000,44.000000,34.164900,-118.273900
max,2.499289e+08,2359.000000,21.000000,120.000000,34.334300,0.000000


In [ ]:
df_categorical = df[features_explore_JA].select_dtypes(include='object')
df_categorical.describe()

,date_report,date_ocurrance,area_name,location,cross_street
count,986500,986500,986500,986500,152270
unique,1749,1749,21,66322,10337
top,02/02/2023 12:00:00 AM,01/01/2020 12:00:00 AM,Central,800 N ALAMEDA ST,BROADWAY
freq,929,1157,68166,2556,2467


In [ ]:
df.loc[df['id_report']<9999999]

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
39890,817,09/20/2020 12:00:00 AM,09/19/2020 12:00:00 AM,1700,17,Devonshire,1777,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,9100 RUBIO AV,NaN,34.2367,-118.4955
268033,2113,06/21/2021 12:00:00 AM,04/05/2021 12:00:00 AM,1835,13,Newton,1385,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,800 E GAGE AV,NaN,33.9820,-118.2622
581052,2203,12/31/2022 12:00:00 AM,12/19/2022 12:00:00 AM,1930,3,Southwest,356,1,522,"VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, B...",...,IC,Invest Cont,522.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
734486,2315,07/23/2023 12:00:00 AM,07/22/2023 12:00:00 AM,1800,15,N Hollywood,1566,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,4400 ENSIGN AV,NaN,34.1504,-118.3712
876571,2401,01/10/2024 12:00:00 AM,12/21/2023 12:00:00 AM,1930,14,Pacific,1416,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3400 CABRILLO BL,NaN,34.0099,-118.4453


In [ ]:
df['age_victim'].loc[df['age_victim']<0]

,age_victim
143,-2
375,-2
581,-1
595,-1
618,-1
...,...
636182,-1
654233,-1
681376,-1
728608,-1


In [ ]:
df[features_explore_JA].